## Multi-Criteria Thermal Site Placement for Greece (Optimized)

This optimized script processes Earth observation data using Google Earth Engine to identify suitable locations for thermal sensor deployment across **all of Greece**. The script has been optimized for better performance, scalability, and includes configurable parameters for easy adjustment.

### Configuration & Imports

In [2]:
import ee
import time
from datetime import datetime
import json

# Configuration for easy parameter adjustment
CONFIG = {
    'start_date': '2015-01-01',
    'end_date': '2024-01-01',
    'scale': 1000,
    'export_scale': 1000,
    'max_pixels': 1e13,
    'cloud_threshold': 57,
    'reduce_scale': 5000,  # Scale for statistical reductions
    'tile_scale': 16,  # For better memory management
}

# Initialize Earth Engine
# ee.Authenticate(force=True)
ee.Authenticate()
ee.Initialize()

### Area of Interest - All of Greece

In [3]:
# Get Greece boundaries from LSIB dataset
greece = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_na', 'Greece')
)

# Get geometry and simplify for better performance
areaOfInterest = greece.geometry().simplify(maxError=1000)

# Validation point 
myLocation = ee.Geometry.Point([22.370536316102555, 37.6525291626026])

# Print area info
area_km2 = areaOfInterest.area().divide(1e6).getInfo()
print(f"Processing area: Greece")
print(f"Total area: {area_km2:,.0f} km²")

Processing area: Greece
Total area: 131,865 km²


### Helper Functions for Optimization

In [4]:
# Helper functions for better performance
def apply_scale_and_mask(collection, scale_factor, band_name):
    """Apply scale factor and mask in one operation"""
    return collection.map(lambda img: 
        img.select(band_name)
           .multiply(scale_factor)
           .updateMask(img.select(band_name).mask())
    )

def create_temporal_composite(collection, reducer='median'):
    """Create temporal composite with specified reducer"""
    if reducer == 'median':
        return collection.median()
    elif reducer == 'mean':
        return collection.mean()
    else:
        return collection.reduce(ee.Reducer.percentile([50]))

def batch_export(exports_list, folder, delay=3):
    """Batch export with rate limiting"""
    tasks = []
    for img, desc in exports_list:
        task = export_image(img, desc, folder)
        if task:
            tasks.append(task)
        time.sleep(delay)
    return tasks

### Land Cover Processing (Optimized)

In [5]:
# Optimized land cover processing
landCover = ee.Image("ESA/WorldCover/v100/2020").clip(areaOfInterest)

# More efficient masking using remap
excluded_classes = [10, 50, 70, 80, 90]
landCoverMask = landCover.remap(
    excluded_classes,
    ee.List.repeat(0, len(excluded_classes)),
    1
).rename('land_cover_mask')

### Albedo Processing (Optimized)

In [6]:
# Optimized albedo with filterBounds for efficiency
albedo_collection = ee.ImageCollection("MODIS/061/MCD43A3") \
    .filterBounds(areaOfInterest) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date']) \
    .select('Albedo_BSA_shortwave')

# Use median for robustness against outliers
albedo = albedo_collection.median().multiply(0.001).clip(areaOfInterest)
albedoMask = albedo.gte(0.122).And(albedo.lte(0.172))

### Cloud Analysis (Optimized)

In [7]:
# Optimized cloud masking
def maskCloudsOptimized(img):
    qa = img.select('state_1km')
    clear = qa.bitwiseAnd(3).eq(0)
    return ee.Image(1).updateMask(clear).set('system:time_start', img.get('system:time_start'))

cloudFree = ee.ImageCollection("MODIS/061/MOD09GA") \
    .filterBounds(areaOfInterest) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date']) \
    .map(maskCloudsOptimized)

# More efficient counting
total_images = cloudFree.size()
cloud_free_count = cloudFree.sum()
cloudFreePercent = cloud_free_count.divide(total_images).multiply(100).clip(areaOfInterest)
cloudMask = cloudFreePercent.gte(CONFIG['cloud_threshold'])

### Terrain Analysis (Optimized)

In [8]:
# Use pre-computed terrain products for efficiency
dem = ee.Image('USGS/SRTMGL1_003').clip(areaOfInterest)
terrain = ee.Algorithms.Terrain(dem)

elevation = terrain.select('elevation')
slope = terrain.select('slope')

# Combined terrain mask
terrainMask = elevation.lte(700).And(slope.lte(8))

### Emissivity Processing (Optimized)

In [9]:
# Optimized emissivity processing
def processEmissivity(img):
    return img.select(['Emis_31', 'Emis_32']) \
              .reduce(ee.Reducer.mean()) \
              .multiply(0.002).add(0.49)

emissivity = ee.ImageCollection('MODIS/061/MOD11A1') \
    .filterBounds(areaOfInterest) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date']) \
    .map(processEmissivity)

# Calculate statistics efficiently
emissivityStats = emissivity.reduce(
    ee.Reducer.mean().combine(
        ee.Reducer.stdDev(), '', True
    )
).clip(areaOfInterest)

# FIXED: Correct band names when using combined reducers
emissivityMean = emissivityStats.select('mean_mean')
emissivityStdDev = emissivityStats.select('mean_stdDev')

emissivityMask = emissivityMean.gt(0.98).And(emissivityStdDev.lt(0.0009))

### Evapotranspiration Processing (Optimized)

In [10]:
# Optimized ET processing
etCollection = ee.ImageCollection('MODIS/061/MOD16A2GF') \
    .filterBounds(areaOfInterest) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date']) \
    .select('ET') \
    .map(lambda img: img.multiply(0.1))

# Combined statistics calculation
etStats = etCollection.reduce(
    ee.Reducer.mean().combine(
        ee.Reducer.stdDev(), '', True
    )
).clip(areaOfInterest)

etMean = etStats.select('ET_mean')
etStdDev = etStats.select('ET_stdDev')

etMask = etMean.gt(8).And(etMean.lt(15)).And(etStdDev.lt(5))

### Water Vapor Processing (Optimized)

In [11]:
# Optimized water vapor processing
waterVapor = ee.ImageCollection('MODIS/061/MCD19A2_GRANULES') \
    .filterBounds(areaOfInterest) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date']) \
    .select('Column_WV') \
    .map(lambda img: img.multiply(0.001))

# Combined statistics
wvStats = waterVapor.reduce(
    ee.Reducer.mean().combine(
        ee.Reducer.stdDev(), '', True
    )
).clip(areaOfInterest)

wvMean = wvStats.select('Column_WV_mean')
wvStdDev = wvStats.select('Column_WV_stdDev')

wvMask = wvMean.lte(1.3).And(wvStdDev.lt(0.7))

### Solar Radiation Analysis (Optimized)

In [12]:
# Highly optimized radiation processing
def processRadiationOptimized(img):
    # Select all DSR bands at once
    dsr_bands = img.select('GMT_.*_DSR')
    
    # Single-pass statistics
    stats = dsr_bands.reduce(
        ee.Reducer.mean().combine(
            ee.Reducer.minMax(), '', True
        )
    ).multiply(0.001)
    
    daily_mean = stats.select('.*mean')
    daily_range = stats.select('.*max').subtract(stats.select('.*min'))
    
    return ee.Image.cat([
        daily_mean.reduce(ee.Reducer.mean()).rename('daily_mean_kw'),
        daily_range.reduce(ee.Reducer.mean()).rename('daily_range_kw')
    ]).copyProperties(img, ['system:time_start'])

# Process radiation
radiationCollection = ee.ImageCollection('MODIS/062/MCD18A1') \
    .filterBounds(areaOfInterest) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date']) \
    .map(processRadiationOptimized)

# Temporal statistics
radiationStats = radiationCollection.select('daily_mean_kw').reduce(
    ee.Reducer.mean().combine(
        ee.Reducer.stdDev(), '', True
    )
).clip(areaOfInterest)

# Annual variability - optimized
def getYearlyRadiation(year):
    return radiationCollection \
        .filter(ee.Filter.calendarRange(year, year, 'year')) \
        .select('daily_mean_kw').mean() \
        .set('year', year)

years = ee.List.sequence(2015, 2023)
yearlyRadiation = ee.ImageCollection(years.map(getYearlyRadiation))
annualRadiationStdDev = yearlyRadiation.reduce(ee.Reducer.stdDev()).clip(areaOfInterest)

# Create mask
radiationMean = radiationStats.select('daily_mean_kw_mean')
radiationStdDev = radiationStats.select('daily_mean_kw_stdDev')

radiationMask = radiationMean.gt(0.19).And(radiationMean.lt(0.7)) \
    .And(radiationStdDev.lt(0.3)).And(annualRadiationStdDev.lt(0.15))

### LST Analysis (Highly Optimized)

In [13]:
# Optimized LST processing with reduced memory footprint
def processLSTOptimized(img):
    lst_day = img.select('LST_Day_1km')
    lst_night = img.select('LST_Night_1km')
    
    # Apply scale and convert in one operation
    day_celsius = lst_day.multiply(0.02).subtract(273.15)
    night_celsius = lst_night.multiply(0.02).subtract(273.15)
    diurnal_range = day_celsius.subtract(night_celsius)
    
    return ee.Image.cat([
        day_celsius.rename('lst_day'),
        night_celsius.rename('lst_night'),
        diurnal_range.rename('diurnal_range')
    ]).copyProperties(img, ['system:time_start'])

# Process LST with bounds filter
modisLST = ee.ImageCollection('MODIS/061/MOD11A1') \
    .filterBounds(areaOfInterest) \
    .filterDate(CONFIG['start_date'], CONFIG['end_date']) \
    .select(['LST_Day_1km', 'LST_Night_1km']) \
    .map(processLSTOptimized)

# Optimized yearly metrics calculation
def calculateYearlyLSTMetrics(year):
    yearly_data = modisLST.filter(ee.Filter.calendarRange(year, year, 'year'))
    
    # All metrics in one image
    return ee.Image.cat([
        yearly_data.select('lst_day').reduce(ee.Reducer.stdDev()).rename('daily_std'),
        yearly_data.select('lst_day').mean().rename('day_mean'),
        yearly_data.select('diurnal_range').mean().rename('diurnal_mean')
    ]).set('year', year)

# Process all years
yearlyMetrics = ee.ImageCollection(years.map(calculateYearlyLSTMetrics))

# Final metrics
meanDailyStdDev = yearlyMetrics.select('daily_std').mean().clip(areaOfInterest)
annualStdDev = yearlyMetrics.select('day_mean').reduce(ee.Reducer.stdDev()).clip(areaOfInterest)
meanDiurnalRange = yearlyMetrics.select('diurnal_mean').mean().clip(areaOfInterest)

# Calculate day and night temperature statistics
dayTempMean = modisLST.select('lst_day').mean().clip(areaOfInterest)
dayTempStdDev = modisLST.select('lst_day').reduce(ee.Reducer.stdDev()).clip(areaOfInterest)
nightTempMean = modisLST.select('lst_night').mean().clip(areaOfInterest)
nightTempStdDev = modisLST.select('lst_night').reduce(ee.Reducer.stdDev()).clip(areaOfInterest)

### LST Masks Creation

In [14]:
# LST thresholds
LST_THRESHOLDS = {
    'DAILY_STD_MIN': 10,
    'DAILY_STD_MAX': 20,
    'ANNUAL_STD': 1.2,
    'DIURNAL_MIN': 15,
    'DIURNAL_MAX': 20
}

# Create masks
dailyStdMask = meanDailyStdDev.gte(LST_THRESHOLDS['DAILY_STD_MIN']) \
    .And(meanDailyStdDev.lte(LST_THRESHOLDS['DAILY_STD_MAX']))

annualStdMask = annualStdDev.gte(LST_THRESHOLDS['ANNUAL_STD'])

diurnalRangeMask = meanDiurnalRange.gte(LST_THRESHOLDS['DIURNAL_MIN']) \
    .And(meanDiurnalRange.lte(LST_THRESHOLDS['DIURNAL_MAX']))

combinedLstMask = dailyStdMask.And(annualStdMask).And(diurnalRangeMask)

### Final Composite Mask

In [15]:
# Combine all masks
extMask = landCoverMask \
    .And(albedoMask) \
    .And(cloudMask) \
    .And(terrainMask) \
    .And(emissivityMask) \
    .And(etMask) \
    .And(wvMask) \
    .And(radiationMask) \
    .And(combinedLstMask) \
    .rename('final_thermal_sites_mask')


### Validation Points

In [16]:
# 1. Define your validation points
validation_points = {
    'Thriasio': ee.Geometry.Point([23.587501971636073, 38.07943033450919]),
    'Attica_East': ee.Geometry.Point([24.02785437776951, 37.89603989461879]),
    'Artemisio': ee.Geometry.Point([22.370536, 37.652529]),
    'Peloponnese_North': ee.Geometry.Point([22.13042611411122, 38.22417894292904]),
    'Peloponnese_Central': ee.Geometry.Point([22.1118, 37.5138]),
    'Peloponnese_South': ee.Geometry.Point([22.4167, 36.7833]),
    'Peloponnese_West': ee.Geometry.Point([21.29587030506639, 37.85110018234032]),
    'Peloponnese_East': ee.Geometry.Point([23.0542, 37.5633])
}

# 2. Create a FeatureCollection with point names as properties
points_fc = ee.FeatureCollection([
    ee.Feature(geometry, {'name': name})
    for name, geometry in validation_points.items()
])

# 3. IMPORTANT: Check and rename your bands to have consistent naming
print("Checking band names...")

# First, let's check what bands each image actually has
def print_band_info(image, image_name):
    """Helper function to print band names"""
    try:
        band_names = image.bandNames().getInfo()
        print(f"{image_name} bands: {band_names}")
    except Exception as e:
        print(f"Error getting bands for {image_name}: {e}")

# Check your image band names (uncomment these lines to debug)
print_band_info(elevation, "elevation")
print_band_info(slope, "slope") 
print_band_info(meanDailyStdDev, "meanDailyStdDev")
print_band_info(annualStdDev, "annualStdDev")
print_band_info(meanDiurnalRange, "meanDiurnalRange")
print_band_info(albedo, "albedo")
print_band_info(cloudFreePercent, "cloudFreePercent")
print_band_info(emissivityMean, "emissivityMean")
print_band_info(etMean, "etMean")
print_band_info(wvMean, "wvMean")

# 4. Create validation image with proper band renaming
# Make sure to add the LST mean and rename bands consistently
validation_image = elevation.select(['elevation'], ['elevation']) \
    .addBands(slope.select([0], ['slope'])) \
    .addBands(meanDailyStdDev.select([0], ['lst_daily_stddev'])) \
    .addBands(annualStdDev.select([0], ['lst_annual_stddev'])) \
    .addBands(meanDiurnalRange.select([0], ['lst_diurnal_range'])) \
    .addBands(albedo.select([0], ['albedo_bsa_shortwave'])) \
    .addBands(cloudFreePercent.select([0], ['cloud_free_percent'])) \
    .addBands(emissivityMean.select([0], ['emissivity_mean'])) \
    .addBands(etMean.select([0], ['et_mean'])) \
    .addBands(wvMean.select([0], ['water_vapor_mean'])) 

# 5. Print final validation image info
print("Final validation image bands:")
try:
    final_bands = validation_image.bandNames().getInfo()
    print(final_bands)
except Exception as e:
    print(f"Error: {e}")

# 6. Sample values at the point locations
sampled_points = validation_image.sampleRegions(
    collection=points_fc,
    scale=1000,
    geometries=True
)

# 7. Export to Google Drive as CSV
task = ee.batch.Export.table.toDrive(
    collection=sampled_points,
    description='Thermal_Validation_Points_Fixed',
    folder='EarthEngine_Exports',
    fileFormat='CSV'
)

# 8. Start the export
task.start()
print("✅ Export started. Check Tasks tab in Earth Engine Code Editor.")



Checking band names...
elevation bands: ['elevation']
slope bands: ['slope']
meanDailyStdDev bands: ['daily_std']
annualStdDev bands: ['day_mean_stdDev']
meanDiurnalRange bands: ['diurnal_mean']
albedo bands: ['Albedo_BSA_shortwave']
cloudFreePercent bands: ['constant']
emissivityMean bands: ['mean_mean']
etMean bands: ['ET_mean']
wvMean bands: ['Column_WV_mean']
Final validation image bands:
['elevation', 'slope', 'lst_daily_stddev', 'lst_annual_stddev', 'lst_diurnal_range', 'albedo_bsa_shortwave', 'cloud_free_percent', 'emissivity_mean', 'et_mean', 'water_vapor_mean']
✅ Export started. Check Tasks tab in Earth Engine Code Editor.


### Optimized Export Function

In [16]:
def export_image(image, description, folder, scale=None):
    """Optimized export with pyramiding and COG"""
    if scale is None:
        scale = CONFIG['export_scale']
    
    # Set default projection for pyramiding
    image = image.setDefaultProjection('EPSG:4326', None, scale)
    
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description[:100],
        folder=folder,
        fileNamePrefix=description[:50],
        region=areaOfInterest,
        scale=scale,
        maxPixels=CONFIG['max_pixels'],
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized': True,
            'noData': -9999
        },
        shardSize=256
    )
    
    task.start()
    print(f"Started: {description}")
    return task

### Export All Products

In [17]:
# Create timestamped folder
export_folder = f'Thermal_Sites_Greece_A{datetime.now().strftime("%Y%m%d")}'

# Organized exports
exports = [
    # ============================================
    # BINARY MASKS (13 total)
    # ============================================
    
    # Final composite mask
    (extMask.selfMask(), 'Final_Thermal_Sites_Mask'),
    
    # Individual parameter masks
    (landCoverMask.selfMask(), 'Land_Cover_Mask'),
    (albedoMask.selfMask(), 'Albedo_Mask'),
    (cloudMask.selfMask(), 'Cloud_Free_Mask'),
    (terrainMask.selfMask(), 'Terrain_Mask'),
    (emissivityMask.selfMask(), 'Emissivity_Mask'),
    (etMask.selfMask(), 'ET_Mask'),
    (wvMask.selfMask(), 'Water_Vapor_Mask'),
    (radiationMask.selfMask(), 'Radiation_Mask'),
    
    # LST masks (individual + combined)
    (dailyStdMask.selfMask(), 'LST_Daily_Mask'),
    (annualStdMask.selfMask(), 'LST_Annual_Mask'),
    (diurnalRangeMask.selfMask(), 'LST_Diurnal_Range_Mask'),
    (combinedLstMask.selfMask(), 'LST_Combined_Mask'),
    
    # ============================================
    # MEAN VALUE PRODUCTS (12 total)
    # ============================================
    
    # Raw/categorical data
    (landCover, 'Land_Cover_Raw'),
    (elevation, 'Elevation'),
    (slope, 'Slope'),
    
    # Environmental means
    (albedo, 'Albedo_Mean'),
    (cloudFreePercent, 'Cloud_Free_Percent'),
    (emissivityMean, 'Emissivity_Mean'),
    (etMean, 'ET_Mean'),
    (wvMean, 'Water_Vapor_Mean'),
    (radiationMean, 'Radiation_Mean'),
    
    # LST means
    (dayTempMean, 'LST_Day_Mean'),
    (nightTempMean, 'LST_Night_Mean'),
    (meanDiurnalRange, 'LST_Diurnal_Range_Mean'),
    
    # ============================================
    # VARIABILITY PRODUCTS (9 total)
    # ============================================
    
    # Environmental standard deviations
    (emissivityStdDev, 'Emissivity_StdDev'),
    (etStdDev, 'ET_StdDev'),
    (wvStdDev, 'Water_Vapor_StdDev'),
    (radiationStdDev, 'Radiation_StdDev'),
    (annualRadiationStdDev, 'Radiation_Annual_StdDev'),
    
    # LST variability metrics
    (dayTempStdDev, 'LST_Day_StdDev'),
    (nightTempStdDev, 'LST_Night_StdDev'),
    (meanDailyStdDev, 'LST_Daily_StdDev'),
    (annualStdDev, 'LST_Annual_StdDev'),
]

# Execute exports with rate limiting
tasks = []
for i, (image, description) in enumerate(exports):
    task = export_image(image, description, export_folder)
    tasks.append(task)
    
    # Progress update every 5 exports
    if (i + 1) % 5 == 0:
        print(f"Progress: {i + 1}/{len(exports)} exports started")
    
    time.sleep(3)  # Rate limiting

print(f"\nAll {len(tasks)} export tasks started!")
print(f"Export folder: {export_folder}")
print("\nCheck Earth Engine Tasks tab for progress.")

Started: Final_Thermal_Sites_Mask
Started: Land_Cover_Mask
Started: Albedo_Mask
Started: Cloud_Free_Mask
Started: Terrain_Mask
Progress: 5/34 exports started
Started: Emissivity_Mask
Started: ET_Mask
Started: Water_Vapor_Mask
Started: Radiation_Mask
Started: LST_Daily_Mask
Progress: 10/34 exports started
Started: LST_Annual_Mask
Started: LST_Diurnal_Range_Mask
Started: LST_Combined_Mask
Started: Land_Cover_Raw
Started: Elevation
Progress: 15/34 exports started
Started: Slope
Started: Albedo_Mean
Started: Cloud_Free_Percent
Started: Emissivity_Mean
Started: ET_Mean
Progress: 20/34 exports started
Started: Water_Vapor_Mean
Started: Radiation_Mean
Started: LST_Day_Mean
Started: LST_Night_Mean
Started: LST_Diurnal_Range_Mean
Progress: 25/34 exports started
Started: Emissivity_StdDev
Started: ET_StdDev
Started: Water_Vapor_StdDev
Started: Radiation_StdDev
Started: Radiation_Annual_StdDev
Progress: 30/34 exports started
Started: LST_Day_StdDev
Started: LST_Night_StdDev
Started: LST_Daily_Std

### Summary Statistics

In [ ]:
# # Create summary report
# summary = {
#     'Analysis': {
#         'Region': 'Greece (entire country)',
#         'Total Area (km²)': f"{area_km2:,.0f}",
#         'Suitable Area (km²)': f"{total_suitable:,.0f}",
#         'Percentage Suitable': f"{percentage:.1f}%",
#         'Date Range': f"{CONFIG['start_date']} to {CONFIG['end_date']}",
#         'Processing Scale': f"{CONFIG['scale']}m",
#         'Export Scale': f"{CONFIG['export_scale']}m"
#     },
#     'Optimizations': [
#         'FilterBounds for reduced data loading',
#         'Combined reducers for single-pass statistics',
#         'Median composites for outlier robustness',
#         'Cloud-optimized GeoTIFF exports',
#         'Pyramiding policy for faster visualization',
#         'Simplified geometry for better performance'
#     ],
#     'Export Info': {
#         'Total Exports': len(tasks),
#         'Folder': export_folder,
#         'Format': 'Cloud Optimized GeoTIFF'
#     }
# }

# # Display summary
# print("\n" + "="*50)
# print("ANALYSIS SUMMARY")
# print("="*50)
# for category, info in summary.items():
#     print(f"\n{category}:")
#     if isinstance(info, dict):
#         for key, value in info.items():
#             print(f"  {key}: {value}")
#     else:
#         for item in info:
#             print(f"  - {item}")

# # Save summary to JSON file
# with open(f'analysis_summary_{datetime.now().strftime("%Y%m%d")}.json', 'w') as f:
#     json.dump(summary, f, indent=2)
# print("\nSummary saved to JSON file.")